In [81]:
mriqc2mrqy('group_T1w.tsv','mod.tsv')

In [80]:
from tsne import bh_sne
import numpy as np

def mriqc2mrqy(input_file, output_file):
    with open(input_file, 'r') as f_input:
        lines = f_input.readlines()

    header = lines[0].strip().split('\t')
    header[0] = "#dataset:Patient"
    header.insert(1, "Name of Images")
    header.insert(2, "NUM") #number of images to show up
    
    #add t-sne (and umap header)
    header.extend(["x", "y"])    

    with open(output_file, 'w') as f_output:
        f_output.write("#start_time:	2021-12-20 09:29:49.078558\n")
        f_output.write("#outdir:	/home/UserInterface/Data/GRIEG\n")
        
        f_output.write('\t'.join(header) +'\n')

        data=[]
        subjids=[]
        for line in lines[1:]:
            values = line.strip().split('\t')
            #values = values[1:] #skip first with subjids
            subjids.append(values[0])
            data.append([float(value) for value in values[1:]])
         
        #calculate tsne coordinates
        array = np.array(data)
        data_tsne = bh_sne(array)
        xs = data_tsne[:, 0]
        ys = data_tsne[:, 1]
        #data=np.column_stack((data,xs,ys))
        
        for subjid, values, x, y in zip(subjids,data,xs,ys):
            #images = f"['{subjid}.png']"
            images = [f"{i}.png" for i in range(1, 43)]
            values.insert(0,images)
            values.insert(1,42) #number of images to show up
            values.insert(0,subjid)
            values.append(str(x))
            values.append(str(y))
            f_output.write('\t'.join(map(str,values)) + '\n')

In [77]:
# Example usage:
html_file = 'sub-91025908_T1w.html'

extract_base64_images_up_to_stop(html_file)

In [78]:
import re
import base64
import os

def extract_base64_images_up_to_stop(html_file):
    output_dir = os.path.splitext(html_file)[0]
    os.makedirs(output_dir, exist_ok=True)
  
    with open(html_file, 'r', encoding='utf-8') as f:
        html_content = f.read()

    # Split the HTML content at the line with '<div class="card-header">Background noise</div>'
    split_content = html_content.split('<div class="card-header">Background noise</div>', 1)
    first_part = split_content[0]

    # Use regular expression to find the base64-encoded image data in the first part
    base64_images = re.findall(r'xlink:href="data:image/(\w+);base64,([^"]+)"', first_part)

    image_counter = 0
    image_id=1
    for image_format, base64_data in base64_images:
        image_bytes = base64.b64decode(base64_data)
        image_counter += 1
        if image_counter % 3 == 0:
            with open(os.path.join(output_dir,f"{image_id}.{image_format}"), 'wb') as f_output:
                f_output.write(image_bytes)
                image_id+=1


In [75]:
file_list = [f"{i}.png" for i in range(1, 43)]

print(file_list)

['1.png', '2.png', '3.png', '4.png', '5.png', '6.png', '7.png', '8.png', '9.png', '10.png', '11.png', '12.png', '13.png', '14.png', '15.png', '16.png', '17.png', '18.png', '19.png', '20.png', '21.png', '22.png', '23.png', '24.png', '25.png', '26.png', '27.png', '28.png', '29.png', '30.png', '31.png', '32.png', '33.png', '34.png', '35.png', '36.png', '37.png', '38.png', '39.png', '40.png', '41.png', '42.png']
